In [1]:
#Modules for executing steps of Notebook
import acquire
import explore

In [ ]:
#Plotting libraries
import warnings
import matplotlib as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
from pylab import rcParams

#Style-settings for Notebook
rcParams['figure.figsize'] = 18, 8
plt.style.use('fivethirtyeight')

In [ ]:
#Statistic tools for plots and for scoring
import statsmodels.api as sm

## ACQUIRE/PREP

Acquire the data from the online source at https://ds.codeup.com/fitbit-data.tar.gz

In [2]:
df = acquire.get_data()

In [ ]:
df.sort_values('Date')

In [3]:
df.Date.nunique()

225

In [ ]:
df.Date.drop_duplicates()

Df2 is the prepared data, with no nulls and with all dtypes are numerical

In [ ]:
df2 = acquire.clean_data(df)

In [ ]:
df

Scaled DataFrame if need be

In [ ]:
scaled_df = acquire.scale_data(df2)

scaled_df = scaled_df.set_index(df2.index)

# EXPLORE

#### Bird's Eye View of the Dataset

In [ ]:
df2.plot()
plt.pyplot.show()

#### Bird's Eye View of the Dataset

#### With scaled data

In [ ]:
scaled_df.plot()
plt.pyplot.show()

The day of the murder!

In [ ]:
day_of_murder = scaled_df[scaled_df['Calories Burned'] == scaled_df['Calories Burned'].min()]

In [ ]:
murder_range = df2['2018-07':'2018-09']

In [ ]:
murder_range.plot()

In [ ]:
pre_murder = df2[:'2018-08-26']

In [ ]:
post_murder = df2['2018-08-26':]

In [ ]:
pre_murder.describe()

In [ ]:
post_murder.describe()

In [ ]:
plt.pyplot.plot(pre_murder)
plt.pyplot.plot(post_murder)
plt.pyplot.show()

## Decomposition

Maybe the trend lines are different for pre and post murder?

# FORECASTING